# Context Optimization and Token Management

This notebook demonstrates advanced context optimization techniques including:
- Chat reducers for conversation management
- Progressive summarization
- Token-efficient context building
- Context window management

## Setup

In [ ]:
import os
import json
from typing import List, Dict, Tuple
from dataclasses import dataclass, field
from datetime import datetime
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential

# Initialize client
github_token = os.environ.get("GITHUB_TOKEN")
if not github_token:
    raise ValueError("GITHUB_TOKEN environment variable must be set")

endpoint = "https://models.github.ai/inference"
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(github_token)
)
model = "gpt-4o-mini"

print(f"✅ Setup complete - Using {model}")

## 1. Token Estimation

Estimate tokens before making API calls.

In [ ]:
def estimate_tokens(text: str) -> int:
    """Rough estimation: ~4 characters per token for English text."""
    return len(text) // 4

def count_message_tokens(messages: List) -> int:
    """Estimate total tokens in a message list."""
    total = 0
    for msg in messages:
        if hasattr(msg, 'content'):
            total += estimate_tokens(msg.content)
        # Add overhead for message structure
        total += 4
    return total

# Test estimation
test_messages = [
    SystemMessage(content="You are a helpful assistant."),
    UserMessage(content="Explain quantum computing in simple terms."),
]

estimated = count_message_tokens(test_messages)
print(f"📊 Estimated tokens: {estimated}")

# Get actual count
response = client.complete(messages=test_messages, model=model)
actual = response.usage.prompt_tokens
print(f"🎯 Actual tokens: {actual}")
print(f"📈 Accuracy: {(1 - abs(estimated - actual) / actual) * 100:.1f}%")

## 2. Chat Reducer - Sliding Window

Keep only the most recent N messages to stay within token limits.

In [ ]:
@dataclass
class Message:
    role: str
    content: str
    tokens: int
    timestamp: datetime = field(default_factory=datetime.now)
    important: bool = False

class SlidingWindowReducer:
    """Keep only recent messages within token budget."""
    
    def __init__(self, max_tokens: int = 4000, window_size: int = 10):
        self.max_tokens = max_tokens
        self.window_size = window_size
    
    def reduce(self, messages: List[Message]) -> List[Message]:
        # Always keep system message
        system_msgs = [m for m in messages if m.role == 'system']
        other_msgs = [m for m in messages if m.role != 'system']
        
        # Keep important messages
        important_msgs = [m for m in other_msgs if m.important]
        
        # Take most recent messages
        recent_msgs = other_msgs[-self.window_size:]
        
        # Combine and deduplicate
        result = system_msgs + important_msgs
        for msg in recent_msgs:
            if msg not in result:
                result.append(msg)
        
        # Check token budget
        total_tokens = sum(m.tokens for m in result)
        while total_tokens > self.max_tokens and len(result) > len(system_msgs) + 1:
            # Remove oldest non-system, non-important message
            for i, msg in enumerate(result):
                if msg.role != 'system' and not msg.important:
                    removed = result.pop(i)
                    total_tokens -= removed.tokens
                    break
        
        return result

# Demo
messages = [
    Message('system', 'You are a helpful assistant.', 50),
    Message('user', 'What is Python?', 20),
    Message('assistant', 'Python is a programming language...', 100),
    Message('user', 'What are decorators?', 25, important=True),
    Message('assistant', 'Decorators are a design pattern...', 150),
    Message('user', 'Show me an example', 30),
    Message('assistant', 'Here is an example: @decorator...', 200),
    Message('user', 'What about generators?', 25),
    Message('assistant', 'Generators are functions that yield...', 180),
]

reducer = SlidingWindowReducer(max_tokens=400, window_size=4)
reduced = reducer.reduce(messages)

print(f"📉 Original: {len(messages)} messages, {sum(m.tokens for m in messages)} tokens")
print(f"📊 Reduced: {len(reduced)} messages, {sum(m.tokens for m in reduced)} tokens")
print(f"\n✅ Kept messages:")
for msg in reduced:
    important_marker = "⭐" if msg.important else "  "
    print(f"{important_marker} {msg.role:10s}: {msg.content[:50]}...")

## 3. Progressive Summarization

Compress old conversation history into concise summaries.

In [ ]:
class ProgressiveSummarizer:
    """Compress conversation history into summaries."""
    
    def __init__(self, client, model):
        self.client = client
        self.model = model
    
    def summarize_conversation(self, messages: List[Message], max_tokens: int = 200) -> str:
        """Create a summary of conversation history."""
        # Build conversation text
        conversation_text = "\n".join([
            f"{msg.role}: {msg.content}" 
            for msg in messages 
            if msg.role in ['user', 'assistant']
        ])
        
        # Request summary
        response = self.client.complete(
            messages=[
                SystemMessage(content=f"""Summarize this conversation in {max_tokens} tokens or less.
Focus on: key topics discussed, decisions made, important information shared.
Format: Brief bullet points."""),
                UserMessage(content=conversation_text)
            ],
            model=self.model,
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    
    def hierarchical_compression(self, text: str, levels: List[int] = [500, 200, 50]) -> List[str]:
        """Create multiple compression levels."""
        summaries = []
        current = text
        
        for max_tokens in levels:
            response = self.client.complete(
                messages=[
                    SystemMessage(content=f"Summarize in {max_tokens} tokens or less. Be concise."),
                    UserMessage(content=current)
                ],
                model=self.model,
                max_tokens=max_tokens
            )
            summary = response.choices[0].message.content
            summaries.append(summary)
            current = summary
        
        return summaries

# Demo
summarizer = ProgressiveSummarizer(client, model)

# Simulate long conversation
long_conversation = [
    Message('user', 'I need to build an authentication system', 50),
    Message('assistant', 'I can help with that. What features do you need?', 60),
    Message('user', 'JWT tokens, refresh tokens, password hashing', 50),
    Message('assistant', 'Great. I recommend using bcrypt for passwords...', 200),
    Message('user', 'How do I implement refresh tokens?', 40),
    Message('assistant', 'Store refresh tokens in a secure database...', 300),
]

summary = summarizer.summarize_conversation(long_conversation, max_tokens=100)

print("📝 CONVERSATION SUMMARY:")
print("="*60)
print(summary)
print("\n" + "="*60)
print(f"Original: ~{sum(m.tokens for m in long_conversation)} tokens")
print(f"Summary: ~{estimate_tokens(summary)} tokens")
print(f"Compression: {(1 - estimate_tokens(summary) / sum(m.tokens for m in long_conversation)) * 100:.1f}%")

## 4. Context Window Manager

Strategically allocate token budget across different context types.

In [ ]:
@dataclass
class ContextBudget:
    system_prompt: int = 500
    conversation_history: int = 1000
    retrieved_context: int = 1500
    tool_outputs: int = 500
    response_reserve: int = 1500
    
    @property
    def total(self) -> int:
        return (self.system_prompt + self.conversation_history + 
                self.retrieved_context + self.tool_outputs + self.response_reserve)

class ContextWindowManager:
    """Manage token allocation across context types."""
    
    def __init__(self, max_tokens: int = 8000):
        self.max_tokens = max_tokens
    
    def allocate(self, priorities: Dict[str, float]) -> ContextBudget:
        """Allocate tokens based on priority weights."""
        # Fixed allocations
        system_tokens = 500
        response_tokens = 1500
        
        # Remaining budget
        remaining = self.max_tokens - system_tokens - response_tokens
        
        # Normalize priorities
        total_priority = sum(priorities.values())
        normalized = {k: v / total_priority for k, v in priorities.items()}
        
        return ContextBudget(
            system_prompt=system_tokens,
            conversation_history=int(remaining * normalized.get('history', 0.3)),
            retrieved_context=int(remaining * normalized.get('retrieved', 0.5)),
            tool_outputs=int(remaining * normalized.get('tools', 0.2)),
            response_reserve=response_tokens
        )
    
    def fit_to_budget(self, content: List[str], budget: int) -> List[str]:
        """Select content that fits within token budget."""
        result = []
        tokens_used = 0
        
        for item in content:
            item_tokens = estimate_tokens(item)
            if tokens_used + item_tokens <= budget:
                result.append(item)
                tokens_used += item_tokens
            else:
                break
        
        return result

# Demo
manager = ContextWindowManager(max_tokens=8000)

# Scenario 1: Code generation (need more examples)
code_gen_budget = manager.allocate({
    'history': 0.2,
    'retrieved': 0.6,  # More examples and docs
    'tools': 0.2
})

print("💻 CODE GENERATION BUDGET:")
print(f"  System: {code_gen_budget.system_prompt} tokens")
print(f"  History: {code_gen_budget.conversation_history} tokens")
print(f"  Retrieved: {code_gen_budget.retrieved_context} tokens")
print(f"  Tools: {code_gen_budget.tool_outputs} tokens")
print(f"  Response: {code_gen_budget.response_reserve} tokens")
print(f"  Total: {code_gen_budget.total} tokens\n")

# Scenario 2: Conversation (need more history)
chat_budget = manager.allocate({
    'history': 0.7,  # More conversation context
    'retrieved': 0.2,
    'tools': 0.1
})

print("💬 CONVERSATION BUDGET:")
print(f"  System: {chat_budget.system_prompt} tokens")
print(f"  History: {chat_budget.conversation_history} tokens")
print(f"  Retrieved: {chat_budget.retrieved_context} tokens")
print(f"  Tools: {chat_budget.tool_outputs} tokens")
print(f"  Response: {chat_budget.response_reserve} tokens")
print(f"  Total: {chat_budget.total} tokens")

## 5. Semantic Caching

Cache responses for semantically similar queries.

In [ ]:
from difflib import SequenceMatcher

class SemanticCache:
    """Simple semantic caching based on string similarity."""
    
    def __init__(self, similarity_threshold: float = 0.85):
        self.cache: Dict[str, Tuple[str, datetime]] = {}
        self.threshold = similarity_threshold
        self.ttl_seconds = 3600  # 1 hour
    
    def _similarity(self, a: str, b: str) -> float:
        """Calculate similarity between two strings."""
        return SequenceMatcher(None, a.lower(), b.lower()).ratio()
    
    def get(self, query: str) -> str | None:
        """Get cached response if similar query exists."""
        now = datetime.now()
        
        for cached_query, (response, timestamp) in self.cache.items():
            # Check TTL
            age = (now - timestamp).total_seconds()
            if age > self.ttl_seconds:
                continue
            
            # Check similarity
            similarity = self._similarity(query, cached_query)
            if similarity >= self.threshold:
                return response
        
        return None
    
    def set(self, query: str, response: str):
        """Cache a query-response pair."""
        self.cache[query] = (response, datetime.now())
    
    def clear_expired(self):
        """Remove expired cache entries."""
        now = datetime.now()
        expired = [
            query for query, (_, timestamp) in self.cache.items()
            if (now - timestamp).total_seconds() > self.ttl_seconds
        ]
        for query in expired:
            del self.cache[query]

# Demo
cache = SemanticCache(similarity_threshold=0.85)

# First query (cache miss)
query1 = "What is machine learning?"
cached = cache.get(query1)

if not cached:
    print("❌ Cache miss - calling API")
    response1 = client.complete(
        messages=[UserMessage(content=query1)],
        model=model
    )
    result1 = response1.choices[0].message.content
    cache.set(query1, result1)
    tokens1 = response1.usage.total_tokens
    print(f"  Used {tokens1} tokens\n")

# Similar query (cache hit)
query2 = "What is machine learning exactly?"
cached = cache.get(query2)

if cached:
    print("✅ Cache hit - using cached response")
    print(f"  Similarity: {cache._similarity(query1, query2):.2%}")
    print(f"  Saved API call!\n")
    result2 = cached
    tokens2 = 0
else:
    print("❌ Cache miss")

# Different query (cache miss)
query3 = "Explain quantum computing"
cached = cache.get(query3)

if not cached:
    print("❌ Cache miss - calling API")
    print(f"  Similarity to query1: {cache._similarity(query1, query3):.2%}")
    print(f"  Below threshold ({cache.threshold:.2%})\n")

print(f"\n📊 Cache Stats:")
print(f"  Entries: {len(cache.cache)}")
print(f"  Hit rate: 1/2 = 50%")
print(f"  Tokens saved: {tokens1}")

## 6. Complete Context Optimization Pipeline

Combining all techniques for production use.

In [ ]:
class OptimizedContextManager:
    """Complete context optimization system."""
    
    def __init__(self, client, model, max_tokens=8000):
        self.client = client
        self.model = model
        self.max_tokens = max_tokens
        
        self.reducer = SlidingWindowReducer(max_tokens=max_tokens)
        self.summarizer = ProgressiveSummarizer(client, model)
        self.window_manager = ContextWindowManager(max_tokens=max_tokens)
        self.cache = SemanticCache()
    
    def process_request(self, 
                       user_query: str,
                       conversation_history: List[Message],
                       retrieved_docs: List[str],
                       system_prompt: str = "You are a helpful assistant.") -> Dict:
        """Process a request with full optimization."""
        
        # 1. Check cache
        cached_response = self.cache.get(user_query)
        if cached_response:
            return {
                'response': cached_response,
                'cached': True,
                'tokens_used': 0
            }
        
        # 2. Reduce conversation history
        reduced_history = self.reducer.reduce(conversation_history)
        
        # 3. Allocate token budget
        budget = self.window_manager.allocate({
            'history': 0.3,
            'retrieved': 0.5,
            'tools': 0.2
        })
        
        # 4. Fit retrieved docs to budget
        fitted_docs = self.window_manager.fit_to_budget(
            retrieved_docs, 
            budget.retrieved_context
        )
        
        # 5. Build context
        messages = [SystemMessage(content=system_prompt)]
        
        # Add conversation summary if history is long
        if len(conversation_history) > 10:
            summary = self.summarizer.summarize_conversation(
                conversation_history[:-5], 
                max_tokens=200
            )
            messages.append(SystemMessage(content=f"Previous conversation summary:\n{summary}"))
        
        # Add recent history
        for msg in reduced_history[-5:]:
            if msg.role == 'user':
                messages.append(UserMessage(content=msg.content))
            elif msg.role == 'assistant':
                messages.append(AssistantMessage(content=msg.content))
        
        # Add retrieved context
        if fitted_docs:
            context = "\n\n".join(fitted_docs)
            messages.append(SystemMessage(content=f"Relevant context:\n{context}"))
        
        # Add current query
        messages.append(UserMessage(content=user_query))
        
        # 6. Make API call
        response = self.client.complete(messages=messages, model=self.model)
        result = response.choices[0].message.content
        
        # 7. Cache response
        self.cache.set(user_query, result)
        
        return {
            'response': result,
            'cached': False,
            'tokens_used': response.usage.total_tokens,
            'history_reduced_from': len(conversation_history),
            'history_reduced_to': len(reduced_history),
            'docs_fitted_from': len(retrieved_docs),
            'docs_fitted_to': len(fitted_docs)
        }

# Demo
manager = OptimizedContextManager(client, model, max_tokens=4000)

# Simulate a request
history = [
    Message('system', 'You are a Python expert.', 50),
    Message('user', 'What are list comprehensions?', 30),
    Message('assistant', 'List comprehensions are...', 100),
    Message('user', 'Show me an example', 25),
    Message('assistant', 'Here is an example...', 150),
]

docs = [
    "Python list comprehensions provide a concise way to create lists.",
    "Syntax: [expression for item in iterable if condition]",
    "List comprehensions are faster than traditional loops.",
]

result = manager.process_request(
    user_query="How do I filter even numbers using list comprehension?",
    conversation_history=history,
    retrieved_docs=docs
)

print("🎯 OPTIMIZED REQUEST RESULT:")
print("="*60)
print(f"Response: {result['response'][:200]}...")
print(f"\nOptimization Stats:")
print(f"  Cached: {result['cached']}")
print(f"  Tokens used: {result['tokens_used']}")
print(f"  History: {result['history_reduced_from']} → {result['history_reduced_to']} messages")
print(f"  Docs: {result['docs_fitted_from']} → {result['docs_fitted_to']} items")
print("="*60)

## Summary

### Key Techniques Demonstrated:

1. **Token Estimation** - Predict costs before API calls
2. **Sliding Window Reducer** - Keep recent + important messages
3. **Progressive Summarization** - Compress old context
4. **Context Window Management** - Strategic token allocation
5. **Semantic Caching** - Avoid redundant API calls
6. **Complete Pipeline** - All techniques combined

### Benefits:

- 📉 **Reduced costs** through token optimization
- ⚡ **Faster responses** via caching
- 🎯 **Better quality** through strategic context
- 🔄 **Scalability** for long conversations

## Next Steps

Explore `05-rag-and-retrieval.ipynb` for retrieval-augmented generation patterns.